### Libs 

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import sys
from os import listdir
from os.path import isfile, join
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from tabulate import tabulate
from tqdm import tqdm

### Dados

In [7]:
FEATURES_SET = {
    "feature": 1,
    "permission": 2,
    "activity": 3,
    "service_receiver": 3,
    "provider": 3,
    "service": 3,
    "intent": 4,
    "api_call": 5,
    "real_permission": 6,
    "call": 7,
    "url": 8
}


def count_feature_set(lines):
    """
    Count how many features belong to a specific set
    :param lines: features in the text file
    :return:
    """
    features_map = {x: 0 for x in range(1, 9)}
    for l in lines:
        if l != "\n":
            set = l.split("::")[0]
            features_map[FEATURES_SET[set]] += 1
    features = []
    for i in range(1, 9):
        features.append(features_map[i])
    return features


In [8]:
def read(LOAD_DATA=False):
    # if LOAD_DATA:
    #     print("Previous data not loaded. Attempt to read data ...")
    #     mypath = r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton\Drebin\MetaData\feature_vectors\feature_vectors"
    #     onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

    #     print("Reading csv file for ground truth ...")
    #     ground_truth = np.loadtxt(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton\Drebin\MetaData\sha256_family.csv", delimiter=",", skiprows=1, dtype=str)
    #     # print ground_truth.shape
    #     # families = np.unique(ground_truth[:, 1])
    #     # print families
    #     # print len(families)

    #     print("Reading positive and negative texts ...")
    #     pos = []
    #     neg = []
    #     for virus in tqdm(onlyfiles):
    #         if virus in ground_truth[:, 0]:
    #             pos.append(virus)
    #         else:
    #             #if len(neg) < 5560:
    #             #if len(neg) < 22240:
    #                 neg.append(virus)

    #     print("Extracting features ...")
    #     x = []
    #     y = []
    #     for text_file in tqdm(pos):
    #         sys.stdin = open("%s/%s" % (mypath, text_file))
    #         features = sys.stdin.readlines()
    #         sample = count_feature_set(features)
    #         x.append(sample)
    #         y.append(1)

    #     for text_file in tqdm(neg):
    #         sys.stdin = open("%s/%s" % (mypath, text_file))
    #         features = sys.stdin.readlines()
    #         sample = count_feature_set(features)
    #         x.append(sample)
    #         y.append(0)

    #     print("Data is read successfully:")
    #     x = np.array(x)
    #     y = np.array(y)
    #     print(x.shape, y.shape)

    #     print("Saving data under data_numpy directory ...")
    #     np.save(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/x_all.npy", x)
    #     np.save(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/y_all.npy", y)

    #     return x, y
    # else:
        print("Loading previous data ...")
        x_ = np.load(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/x_all.npy")
        y_ = np.load(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/y_all.npy")
        print(x_.shape, y_.shape)
        # print x == x_, y == y_
        return x_, y_


def map_family_to_category(families):
    out = {}
    count = 1
    for family in families:
        out[family] = count
        count += 1
    return out


if __name__ == "__main__":
    x, y = read(LOAD_DATA=True)

Loading previous data ...
(129013, 8) (129013,)


In [9]:
x_all, y_all = read(LOAD_DATA=False)
x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, stratify=y_all)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

Loading previous data ...
(129013, 8) (129013,)
(103210, 8) (103210,)
(25803, 8) (25803,)


In [10]:
# # Contagem de classes antes da divisão
# y_all_counts = pd.Series(y_all).value_counts(normalize=True)

# # Divisão dos dados em treinamento e teste
# x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, random_state=42, stratify=y_all)

# # Contagem de classes após a divisão
# y_train_counts = pd.Series(y_train).value_counts(normalize=True)
# y_test_counts = pd.Series(y_test).value_counts(normalize=True)

# # Verificação se as proporções foram mantidas
# print("Proporções das classes no conjunto original:")
# print(y_all_counts)
# print("\nProporções das classes no conjunto de treinamento:")
# print(y_train_counts)
# print("\nProporções das classes no conjunto de teste:")
# print(y_test_counts)

In [11]:
####### DP = Desvio Padrão; PosBal = Pos Balanceamento
Acurracy = []
Precision = []
Recall = []
F1_score = []
Roc_auc = []
######
AcurracyDP = []
PrecisionDP = []
RecallDP = []
F1_scoreDP = []
Roc_aucDP = []
######
AcurracyBagging = []
PrecisionBagging = []
RecallBagging = []
F1Bagging = []
Roc_aucBagging = []
######
AcurracyBaggingDP = []
PrecisionBaggingDP = []
RecallBaggingDP = []
F1BaggingDP = []
Roc_aucBaggingDP = []
#####
AcurracyPosBal = []
PrecisionPosBal = []
RecallPosBal = []
F1_scorePosBal = []
Roc_aucPosBal = []
######
AcurracyDPPosBal = []
PrecisionDPPosBal = []
RecallDPPosBal = []
F1_scoreDPPosBal = []
Roc_aucDPPosBal = []
######
AcurracyBaggingPosBal = []
PrecisionBaggingPosBal = []
RecallBaggingPosBal = []
F1BaggingPosBal = []
Roc_aucBaggingPosBal = []
######
AcurracyBaggingDPPosBal = []
PrecisionBaggingDPPosBal = []
RecallBaggingDPPosBal = []
F1BaggingDPPosBal = []
Roc_aucBaggingDPPosBal = []

### Modelos Base

#### Decision Tree

In [12]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, stratify=y_all)

    # Inicializar o modelo de árvore de decisão com Gini Impurity e sem limitação de altura
    modelo_arvore = DecisionTreeClassifier(criterion='gini')

    # Treinar o modelo no conjunto de treino
    modelo_arvore.fit(x_train, y_train)

    # Fazer previsões no conjunto de teste
    previsoes = modelo_arvore.predict(x_test)

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
Acurracy.append(media_acuracias)
Precision.append(media_precisoes)
Recall.append(media_recalls)
F1_score.append(media_f1_scores)
Roc_auc.append(media_roc_auc_scores)

AcurracyDP.append(dp_acuracias)
PrecisionDP.append(dp_precisoes)
RecallDP.append(dp_recalls)
F1_scoreDP.append(dp_f1_scores)
Roc_aucDP.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  3%|▎         | 1/30 [00:00<00:08,  3.44it/s]

Repetição 1 - Acurácia: 0.98, Precisão: 0.78, Recall: 0.84, F1-score: 0.81, ROC AUC: 0.92


  7%|▋         | 2/30 [00:00<00:07,  3.77it/s]

Repetição 2 - Acurácia: 0.98, Precisão: 0.78, Recall: 0.82, F1-score: 0.80, ROC AUC: 0.91


 10%|█         | 3/30 [00:00<00:06,  3.93it/s]

...

Repetição 28 - Acurácia: 0.98, Precisão: 0.78, Recall: 0.80, F1-score: 0.79, ROC AUC: 0.90


 97%|█████████▋| 29/30 [00:07<00:00,  4.16it/s]

Repetição 29 - Acurácia: 0.98, Precisão: 0.78, Recall: 0.82, F1-score: 0.80, ROC AUC: 0.91


100%|██████████| 30/30 [00:07<00:00,  4.10it/s]

Repetição 30 - Acurácia: 0.98, Precisão: 0.79, Recall: 0.82, F1-score: 0.80, ROC AUC: 0.91
Média das Acurácias: 0.98173 ± 0.00071
Média das Precisões: 0.77117 ± 0.01049
Média dos Recalls: 0.81939 ± 0.01203
Média dos F1-scores: 0.79447 ± 0.00772
Média dos ROC AUC: 0.90422 ± 0.00593


#### Naive Bayes

In [13]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, stratify=y_all)
    
    #Inicializar o modelo de Naive Bayes com distribuição de Bernoulli
    modelo_naive_bayes = GaussianNB()

    # Treinar o modelo no conjunto de treino
    modelo_naive_bayes.fit(x_train, y_train)

    # Fazer previsões no conjunto de teste
    previsoes = modelo_naive_bayes.predict(x_test)

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
Acurracy.append(media_acuracias)
Precision.append(media_precisoes)
Recall.append(media_recalls)
F1_score.append(media_f1_scores)
Roc_auc.append(media_roc_auc_scores)

AcurracyDP.append(dp_acuracias)
PrecisionDP.append(dp_precisoes)
RecallDP.append(dp_recalls)
F1_scoreDP.append(dp_f1_scores)
Roc_aucDP.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  7%|▋         | 2/30 [00:00<00:01, 15.50it/s]

Repetição 1 - Acurácia: 0.91, Precisão: 0.25, Recall: 0.49, F1-score: 0.33, ROC AUC: 0.71
Repetição 2 - Acurácia: 0.92, Precisão: 0.25, Recall: 0.50, F1-score: 0.34, ROC AUC: 0.72
Repetição 3 - Acurácia: 0.91, Precisão: 0.23, Recall: 0.45, F1-score: 0.31, ROC AUC: 0.69
Repetição 4 - Acurácia: 0.92, Precisão: 0.25, Recall: 0.46, F1-score: 0.32, ROC AUC: 0.70


 13%|█▎        | 4/30 [00:00<00:01, 15.56it/s]

Repetição 5 - Acurácia: 0.91, Precisão: 0.24, Recall: 0.47, F1-score: 0.32, ROC AUC: 0.70
Repetição 6 - Acurácia: 0.91, Precisão: 0.23, Recall: 0.44, F1-score: 0.31, ROC AUC: 0.69


 20%|██        | 6/30 [00:00<00:01, 15.59it/s]

...

Repetição 27 - Acurácia: 0.91, Precisão: 0.23, Recall: 0.46, F1-score: 0.31, ROC AUC: 0.69
Repetição 28 - Acurácia: 0.92, Precisão: 0.25, Recall: 0.46, F1-score: 0.32, ROC AUC: 0.70


100%|██████████| 30/30 [00:01<00:00, 15.41it/s]

Repetição 29 - Acurácia: 0.92, Precisão: 0.24, Recall: 0.46, F1-score: 0.32, ROC AUC: 0.70
Repetição 30 - Acurácia: 0.92, Precisão: 0.26, Recall: 0.47, F1-score: 0.33, ROC AUC: 0.70


100%|██████████| 30/30 [00:01<00:00, 15.73it/s]

Média das Acurácias: 0.91629 ± 0.00298
Média das Precisões: 0.25006 ± 0.00980
Média dos Recalls: 0.47035 ± 0.01486
Média dos F1-scores: 0.32638 ± 0.00967
Média dos ROC AUC: 0.70336 ± 0.00703


####  MLP

In [14]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, stratify=y_all)
    
    # Inicializar o modelo MLP
    modelo_mlp = MLPClassifier(hidden_layer_sizes=(100), max_iter=1000)

    # Treinar o modelo no conjunto de treino
    modelo_mlp.fit(x_train, y_train)

    # Fazer previsões no conjunto de teste
    previsoes = modelo_mlp.predict(x_test)

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
Acurracy.append(media_acuracias)
Precision.append(media_precisoes)
Recall.append(media_recalls)
F1_score.append(media_f1_scores)
Roc_auc.append(media_roc_auc_scores)

AcurracyDP.append(dp_acuracias)
PrecisionDP.append(dp_precisoes)
RecallDP.append(dp_recalls)
F1_scoreDP.append(dp_f1_scores)
Roc_aucDP.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  3%|▎         | 1/30 [00:25<12:11, 25.21s/it]

Repetição 1 - Acurácia: 0.98, Precisão: 0.79, Recall: 0.58, F1-score: 0.67, ROC AUC: 0.79


  7%|▋         | 2/30 [00:56<13:21, 28.61s/it]

Repetição 2 - Acurácia: 0.98, Precisão: 0.86, Recall: 0.57, F1-score: 0.68, ROC AUC: 0.78


 10%|█         | 3/30 [01:34<14:53, 33.10s/it]

...

Repetição 28 - Acurácia: 0.98, Precisão: 0.83, Recall: 0.59, F1-score: 0.69, ROC AUC: 0.79


 97%|█████████▋| 29/30 [18:45<00:38, 38.64s/it]

Repetição 29 - Acurácia: 0.98, Precisão: 0.84, Recall: 0.60, F1-score: 0.70, ROC AUC: 0.80


100%|██████████| 30/30 [19:20<00:00, 38.69s/it]

Repetição 30 - Acurácia: 0.98, Precisão: 0.79, Recall: 0.62, F1-score: 0.70, ROC AUC: 0.81
Média das Acurácias: 0.97690 ± 0.00095
Média das Precisões: 0.81707 ± 0.04075
Média dos Recalls: 0.60309 ± 0.04099
Média dos F1-scores: 0.69179 ± 0.01578
Média dos ROC AUC: 0.79841 ± 0.01950


#### KNN

In [15]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, stratify=y_all)
    
    #inicializar o modelo KNN
    modelo_knn = KNeighborsClassifier(n_neighbors=7)  # Escolha o número de vizinhos desejado

    # Treinar o modelo no conjunto de treino
    modelo_knn.fit(np.array(x_train), y_train)

    # Fazer previsões no conjunto de teste
    previsoes = modelo_knn.predict(np.array(x_test))

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
Acurracy.append(media_acuracias)
Precision.append(media_precisoes)
Recall.append(media_recalls)
F1_score.append(media_f1_scores)
Roc_auc.append(media_roc_auc_scores)

AcurracyDP.append(dp_acuracias)
PrecisionDP.append(dp_precisoes)
RecallDP.append(dp_recalls)
F1_scoreDP.append(dp_f1_scores)
Roc_aucDP.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  3%|▎         | 1/30 [00:01<00:51,  1.79s/it]

Repetição 1 - Acurácia: 0.98, Precisão: 0.81, Recall: 0.69, F1-score: 0.75, ROC AUC: 0.84


  7%|▋         | 2/30 [00:03<00:49,  1.77s/it]

Repetição 2 - Acurácia: 0.98, Precisão: 0.80, Recall: 0.73, F1-score: 0.76, ROC AUC: 0.86


 10%|█         | 3/30 [00:05<00:47,  1.75s/it]

...

Repetição 28 - Acurácia: 0.98, Precisão: 0.82, Recall: 0.73, F1-score: 0.77, ROC AUC: 0.86


 97%|█████████▋| 29/30 [00:46<00:01,  1.55s/it]

Repetição 29 - Acurácia: 0.98, Precisão: 0.80, Recall: 0.73, F1-score: 0.76, ROC AUC: 0.86


100%|██████████| 30/30 [00:48<00:00,  1.62s/it]

Repetição 30 - Acurácia: 0.98, Precisão: 0.80, Recall: 0.75, F1-score: 0.77, ROC AUC: 0.87
Média das Acurácias: 0.98119 ± 0.00075
Média das Precisões: 0.81119 ± 0.01283
Média dos Recalls: 0.73471 ± 0.01448
Média dos F1-scores: 0.77093 ± 0.00948
Média dos ROC AUC: 0.86350 ± 0.00715


#### Resultados

In [16]:
import pandas as pd

# Definir os modelos
Modelos = ['Decision Tree', 'Naive Bayes', 'MLP', 'KNN']

# Criar um DataFrame do Pandas com os valores das variáveis
df = pd.DataFrame({
    'Modelos': Modelos,
    'Acuracia (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Acurracy, AcurracyDP)],
    'Precisão (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Precision, PrecisionDP)],
    'Recall (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Recall, RecallDP)],
    'F1 Score': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(F1_score, F1_scoreDP)],
    'AUC RoC': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Roc_auc, Roc_aucDP)]
})

# Função para destacar o maior valor de cada coluna
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

# Aplicar a função de destaque apenas para as colunas numéricas
numeric_columns = df.columns[1:]
styled_df = df.style.apply(highlight_max, subset=numeric_columns)

# Definir a cor do texto para a tabela
styled_df.set_properties(**{'color': 'white'}, subset=pd.IndexSlice[:, :])

# Exibir o DataFrame estilizado no Jupyter Notebook
styled_df


,Modelos,Acuracia (DREBIN),Precisão (DREBIN),Recall (DREBIN),F1 Score,AUC RoC
0,Decision Tree,0.9817 ± 0.0007,0.7712 ± 0.0105,0.8194 ± 0.0120,0.7945 ± 0.0077,0.9042 ± 0.0059
1,Naive Bayes,0.9163 ± 0.0030,0.2501 ± 0.0098,0.4704 ± 0.0149,0.3264 ± 0.0097,0.7034 ± 0.0070
2,MLP,0.9769 ± 0.0009,0.8171 ± 0.0407,0.6031 ± 0.0410,0.6918 ± 0.0158,0.7984 ± 0.0195
3,KNN,0.9812 ± 0.0007,0.8112 ± 0.0128,0.7347 ± 0.0145,0.7709 ± 0.0095,0.8635 ± 0.0071


### Bagging

#### Decision Tree

In [17]:
# Definir o classificador Decision Tree
modelo_arvore = DecisionTreeClassifier(criterion='gini')

# Definir o classificador de Bagging com Decision Tree como base
bagging_clf = BaggingClassifier(estimator = modelo_arvore, n_estimators=30)

# Treinar o modelo Bagging
bagging_clf.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = bagging_clf.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com Decision Tree:", accuracy_bagging)
print("Precisão do modelo Bagging com Decision Tree:", precision_bagging)
print("Recall do modelo Bagging com Decision Tree:", recall_bagging)
print("F1-score do modelo Bagging com Decision Tree:", f1_score_bagging)
print("AUC ROC do modelo Bagging com Decision Tree:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com Decision Tree:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBagging.append(accuracy_bagging)
PrecisionBagging.append(precision_bagging)
RecallBagging.append(recall_bagging)
F1Bagging.append(f1_score_bagging)
Roc_aucBagging.append(auc_roc_bagging)



Acurácia do modelo Bagging com Decision Tree: 0.9876758516451576
Precisão do modelo Bagging com Decision Tree: 0.8993963782696177
Recall do modelo Bagging com Decision Tree: 0.8039568345323741
F1-score do modelo Bagging com Decision Tree: 0.8490028490028491
AUC ROC do modelo Bagging com Decision Tree: 0.8999533879032614
Matriz de confusão do modelo Bagging com Decision Tree:
[[24591   100]
 [  218   894]]


#### Naive Bayes

In [18]:
# Inicializar o modelo de Naive Bayes com distribuição de Bernoulli
modelo_naive_bayes = GaussianNB()

# Definir o classificador de Bagging com Naive Bayes como base
bagging_clf = BaggingClassifier(estimator=modelo_naive_bayes, n_estimators=30)

# Treinar o modelo Bagging
bagging_clf.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = bagging_clf.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com Naive Bayes:", accuracy_bagging)
print("Precisão do modelo Bagging com Naive Bayes:", precision_bagging)
print("Recall do modelo Bagging com Naive Bayes:", recall_bagging)
print("F1-score do modelo Bagging com Naive Bayes:", f1_score_bagging)
print("AUC ROC do modelo Bagging com Naive Bayes:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com Naive Bayes:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBagging.append(accuracy_bagging)
PrecisionBagging.append(precision_bagging)
RecallBagging.append(recall_bagging)
F1Bagging.append(f1_score_bagging)
Roc_aucBagging.append(auc_roc_bagging)

Acurácia do modelo Bagging com Naive Bayes: 0.9178002557842111
Precisão do modelo Bagging com Naive Bayes: 0.24862979571499752
Recall do modelo Bagging com Naive Bayes: 0.4487410071942446
F1-score do modelo Bagging com Naive Bayes: 0.3199743507534466
AUC ROC do modelo Bagging com Naive Bayes: 0.6938330608042018
Matriz de confusão do modelo Bagging com Naive Bayes:
[[23183  1508]
 [  613   499]]


#### MLP

In [19]:
# Definir o classificador base (MLP)
modelo_mlp = MLPClassifier(hidden_layer_sizes=(100), max_iter=1000)

# Definir o classificador de Bagging com MLP como base
bagging_clf = BaggingClassifier(estimator=modelo_mlp, n_estimators=30)

# Treinar o modelo Bagging
bagging_clf.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = bagging_clf.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com MLP:", accuracy_bagging)
print("Precisão do modelo Bagging com MLP:", precision_bagging)
print("Recall do modelo Bagging com MLP:", recall_bagging)
print("F1-score do modelo Bagging com MLP:", f1_score_bagging)
print("AUC ROC do modelo Bagging com MLP:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com MLP:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBagging.append(accuracy_bagging)
PrecisionBagging.append(precision_bagging)
RecallBagging.append(recall_bagging)
F1Bagging.append(f1_score_bagging)
Roc_aucBagging.append(auc_roc_bagging)

Acurácia do modelo Bagging com MLP: 0.9770181761810642
Precisão do modelo Bagging com MLP: 0.8280657395701644
Recall do modelo Bagging com MLP: 0.5890287769784173
F1-score do modelo Bagging com MLP: 0.6883867577509196
AUC ROC do modelo Bagging com MLP: 0.7917603485556296
Matriz de confusão do modelo Bagging com MLP:
[[24555   136]
 [  457   655]]


#### KNN

In [20]:
#inicializar o modelo KNN
modelo_knn = KNeighborsClassifier(n_neighbors=7)  # Escolha o número de vizinhos desejado

# Definir o classificador de Bagging com KNN como base
bagging_clf = BaggingClassifier(estimator=modelo_knn, n_estimators=30)

# Treinar o modelo Bagging
bagging_clf.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = bagging_clf.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com KNN:", accuracy_bagging)
print("Precisão do modelo Bagging com KNN:", precision_bagging)
print("Recall do modelo Bagging com KNN:", recall_bagging)
print("F1-score do modelo Bagging com KNN:", f1_score_bagging)
print("AUC ROC do modelo Bagging com KNN:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com KNN:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBagging.append(accuracy_bagging)
PrecisionBagging.append(precision_bagging)
RecallBagging.append(recall_bagging)
F1Bagging.append(f1_score_bagging)
Roc_aucBagging.append(auc_roc_bagging)

Acurácia do modelo Bagging com KNN: 0.9815912878347479
Precisão do modelo Bagging com KNN: 0.8188188188188188
Recall do modelo Bagging com KNN: 0.7356115107913669
F1-score do modelo Bagging com KNN: 0.7749881572714353
AUC ROC do modelo Bagging com KNN: 0.864140452248788
Matriz de confusão do modelo Bagging com KNN:
[[24510   181]
 [  294   818]]


#### Resultados

In [21]:
Modelos = ['Decision Tree', 'Naive Bayes', 'MLP', 'KNN']
# Criar um DataFrame do Pandas com os valores das variáveis
dfB = pd.DataFrame({'Modelos Bagging': Modelos, 'Acuracia (DREBIN)': AcurracyBagging,
                    'Precisão (DREBIN)': PrecisionBagging, 'Recall (DREBIN)':RecallBagging, 
                    'F1 Score': F1Bagging, 'AUC ROC': Roc_aucBagging})

print(tabulate(dfB, headers='keys', tablefmt='fancy_grid', floatfmt=".4f"))

╒════╤═══════════════════╤═════════════════════╤═════════════════════╤═══════════════════╤════════════╤═══════════╕
│    │ Modelos Bagging   │   Acuracia (DREBIN) │   Precisão (DREBIN) │   Recall (DREBIN) │   F1 Score │   AUC ROC │
╞════╪═══════════════════╪═════════════════════╪═════════════════════╪═══════════════════╪════════════╪═══════════╡
│  0 │ Decision Tree     │              0.9877 │              0.8994 │            0.8040 │     0.8490 │    0.9000 │
├────┼───────────────────┼─────────────────────┼─────────────────────┼───────────────────┼────────────┼───────────┤
│  1 │ Naive Bayes       │              0.9178 │              0.2486 │            0.4487 │     0.3200 │    0.6938 │
├────┼───────────────────┼─────────────────────┼─────────────────────┼───────────────────┼────────────┼───────────┤
│  2 │ MLP               │              0.9770 │              0.8281 │            0.5890 │     0.6884 │    0.7918 │
├────┼───────────────────┼─────────────────────┼─────────────────────┼──

### Balanceamento de dados

In [22]:
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(x_all, y_all)

x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, stratify=y_resampled)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(197524, 8) (197524,)
(49382, 8) (49382,)


In [23]:
# Calcular proporção das classes em y_all
unique_classes, class_counts = np.unique(y_all, return_counts=True)
proporcao_classes_resampled = class_counts / len(y_all)

print("Proporção das classes no conjunto original:")
for classe, proporcao in zip(unique_classes, proporcao_classes_resampled):
    print(f"Classe {classe}: {proporcao:.2f}")

# Calcular proporção das classes em y_resampled
unique_classes, class_counts = np.unique(y_resampled, return_counts=True)
proporcao_classes_resampled = class_counts / len(y_resampled)

print("Proporção das classes no conjunto resampleado:")
for classe, proporcao in zip(unique_classes, proporcao_classes_resampled):
    print(f"Classe {classe}: {proporcao:.2f}")

Proporção das classes no conjunto original:
Classe 0: 0.96
Classe 1: 0.04
Proporção das classes no conjunto resampleado:
Classe 0: 0.50
Classe 1: 0.50


### Modelos Base após o Balanceamento

#### Decision Tree

In [24]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):
    x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, stratify=y_resampled)
    # Inicializar o modelo de árvore de decisão com Gini Impurity e sem limitação de altura
    modelo_arvore = DecisionTreeClassifier(criterion='gini')

    # Treinar o modelo no conjunto de treino
    modelo_arvore.fit(x_train, y_train)

    # Fazer previsões no conjunto de teste
    previsoes = modelo_arvore.predict(x_test)

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
AcurracyPosBal.append(media_acuracias)
PrecisionPosBal.append(media_precisoes)
RecallPosBal.append(media_recalls)
F1_scorePosBal.append(media_f1_scores)
Roc_aucPosBal.append(media_roc_auc_scores)

AcurracyDPPosBal.append(dp_acuracias)
PrecisionDPPosBal.append(dp_precisoes)
RecallDPPosBal.append(dp_recalls)
F1_scoreDPPosBal.append(dp_f1_scores)
Roc_aucDPPosBal.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  3%|▎         | 1/30 [00:00<00:15,  1.89it/s]

Repetição 1 - Acurácia: 0.98, Precisão: 0.97, Recall: 0.99, F1-score: 0.98, ROC AUC: 0.98


  7%|▋         | 2/30 [00:01<00:14,  1.90it/s]

Repetição 2 - Acurácia: 0.98, Precisão: 0.98, Recall: 0.99, F1-score: 0.98, ROC AUC: 0.98


 10%|█         | 3/30 [00:01<00:14,  1.90it/s]

...

Repetição 28 - Acurácia: 0.98, Precisão: 0.98, Recall: 0.99, F1-score: 0.98, ROC AUC: 0.98


 97%|█████████▋| 29/30 [00:15<00:00,  1.90it/s]

Repetição 29 - Acurácia: 0.98, Precisão: 0.97, Recall: 0.99, F1-score: 0.98, ROC AUC: 0.98


100%|██████████| 30/30 [00:15<00:00,  1.89it/s]

Repetição 30 - Acurácia: 0.98, Precisão: 0.97, Recall: 0.99, F1-score: 0.98, ROC AUC: 0.98
Média das Acurácias: 0.98104 ± 0.00065
Média das Precisões: 0.97373 ± 0.00115
Média dos Recalls: 0.98875 ± 0.00083
Média dos F1-scores: 0.98118 ± 0.00064
Média dos ROC AUC: 0.98104 ± 0.00065


#### Naive Bayes

In [25]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, stratify=y_resampled)
    
    #Inicializar o modelo de Naive Bayes com distribuição de Bernoulli
    modelo_naive_bayes = GaussianNB()

    # Treinar o modelo no conjunto de treino
    modelo_naive_bayes.fit(x_train, y_train)

    # Fazer previsões no conjunto de teste
    previsoes = modelo_naive_bayes.predict(x_test)

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
AcurracyPosBal.append(media_acuracias)
PrecisionPosBal.append(media_precisoes)
RecallPosBal.append(media_recalls)
F1_scorePosBal.append(media_f1_scores)
Roc_aucPosBal.append(media_roc_auc_scores)

AcurracyDPPosBal.append(dp_acuracias)
PrecisionDPPosBal.append(dp_precisoes)
RecallDPPosBal.append(dp_recalls)
F1_scoreDPPosBal.append(dp_f1_scores)
Roc_aucDPPosBal.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  3%|▎         | 1/30 [00:00<00:04,  6.53it/s]

Repetição 1 - Acurácia: 0.73, Precisão: 0.73, Recall: 0.74, F1-score: 0.73, ROC AUC: 0.73


  7%|▋         | 2/30 [00:00<00:03,  7.06it/s]

Repetição 2 - Acurácia: 0.73, Precisão: 0.73, Recall: 0.74, F1-score: 0.73, ROC AUC: 0.73


 10%|█         | 3/30 [00:00<00:03,  7.27it/s]

...

Repetição 28 - Acurácia: 0.73, Precisão: 0.72, Recall: 0.74, F1-score: 0.73, ROC AUC: 0.73


 97%|█████████▋| 29/30 [00:03<00:00,  7.38it/s]

Repetição 29 - Acurácia: 0.74, Precisão: 0.74, Recall: 0.74, F1-score: 0.74, ROC AUC: 0.74


100%|██████████| 30/30 [00:04<00:00,  7.43it/s]

Repetição 30 - Acurácia: 0.73, Precisão: 0.73, Recall: 0.74, F1-score: 0.73, ROC AUC: 0.73
Média das Acurácias: 0.72989 ± 0.00460
Média das Precisões: 0.72553 ± 0.00473
Média dos Recalls: 0.73969 ± 0.01407
Média dos F1-scores: 0.73245 ± 0.00672
Média dos ROC AUC: 0.72989 ± 0.00460


#### MLP

In [26]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, stratify=y_resampled)
    
    # Inicializar o modelo MLP
    modelo_mlp = MLPClassifier(hidden_layer_sizes=(100), max_iter=1000)

    # Treinar o modelo no conjunto de treino
    modelo_mlp.fit(x_train, y_train)

    # Fazer previsões no conjunto de teste
    previsoes = modelo_mlp.predict(x_test)

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
AcurracyPosBal.append(media_acuracias)
PrecisionPosBal.append(media_precisoes)
RecallPosBal.append(media_recalls)
F1_scorePosBal.append(media_f1_scores)
Roc_aucPosBal.append(media_roc_auc_scores)

AcurracyDPPosBal.append(dp_acuracias)
PrecisionDPPosBal.append(dp_precisoes)
RecallDPPosBal.append(dp_recalls)
F1_scoreDPPosBal.append(dp_f1_scores)
Roc_aucDPPosBal.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  3%|▎         | 1/30 [01:43<49:47, 103.03s/it]

Repetição 1 - Acurácia: 0.93, Precisão: 0.92, Recall: 0.95, F1-score: 0.94, ROC AUC: 0.93


  7%|▋         | 2/30 [03:13<44:42, 95.81s/it] 

Repetição 2 - Acurácia: 0.93, Precisão: 0.92, Recall: 0.95, F1-score: 0.94, ROC AUC: 0.93


 10%|█         | 3/30 [04:37<40:37, 90.27s/it]

...

Repetição 28 - Acurácia: 0.94, Precisão: 0.93, Recall: 0.94, F1-score: 0.94, ROC AUC: 0.94


 97%|█████████▋| 29/30 [46:34<01:26, 86.61s/it]

Repetição 29 - Acurácia: 0.94, Precisão: 0.92, Recall: 0.95, F1-score: 0.94, ROC AUC: 0.94


100%|██████████| 30/30 [47:38<00:00, 95.29s/it]

Repetição 30 - Acurácia: 0.93, Precisão: 0.93, Recall: 0.94, F1-score: 0.93, ROC AUC: 0.93
Média das Acurácias: 0.93477 ± 0.00278
Média das Precisões: 0.93293 ± 0.00851
Média dos Recalls: 0.93709 ± 0.01225
Média dos F1-scores: 0.93490 ± 0.00317
Média dos ROC AUC: 0.93477 ± 0.00278


#### KNN

In [27]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, stratify=y_resampled)
    
    #inicializar o modelo KNN
    modelo_knn = KNeighborsClassifier(n_neighbors=7)  # Escolha o número de vizinhos desejado

    # Treinar o modelo no conjunto de treino
    modelo_knn.fit(np.array(x_train), y_train)

    # Fazer previsões no conjunto de teste
    previsoes = modelo_knn.predict(np.array(x_test))

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
AcurracyPosBal.append(media_acuracias)
PrecisionPosBal.append(media_precisoes)
RecallPosBal.append(media_recalls)
F1_scorePosBal.append(media_f1_scores)
Roc_aucPosBal.append(media_roc_auc_scores)

AcurracyDPPosBal.append(dp_acuracias)
PrecisionDPPosBal.append(dp_precisoes)
RecallDPPosBal.append(dp_recalls)
F1_scoreDPPosBal.append(dp_f1_scores)
Roc_aucDPPosBal.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  3%|▎         | 1/30 [00:04<02:10,  4.51s/it]

Repetição 1 - Acurácia: 0.97, Precisão: 0.95, Recall: 0.99, F1-score: 0.97, ROC AUC: 0.97


  7%|▋         | 2/30 [00:08<02:05,  4.48s/it]

Repetição 2 - Acurácia: 0.97, Precisão: 0.95, Recall: 0.99, F1-score: 0.97, ROC AUC: 0.97


 10%|█         | 3/30 [00:12<01:55,  4.27s/it]

...

Repetição 28 - Acurácia: 0.97, Precisão: 0.95, Recall: 0.99, F1-score: 0.97, ROC AUC: 0.97


 97%|█████████▋| 29/30 [01:55<00:03,  3.89s/it]

Repetição 29 - Acurácia: 0.97, Precisão: 0.95, Recall: 0.99, F1-score: 0.97, ROC AUC: 0.97


100%|██████████| 30/30 [01:59<00:00,  3.99s/it]

Repetição 30 - Acurácia: 0.97, Precisão: 0.95, Recall: 0.99, F1-score: 0.97, ROC AUC: 0.97
Média das Acurácias: 0.97081 ± 0.00098
Média das Precisões: 0.95170 ± 0.00165
Média dos Recalls: 0.99197 ± 0.00137
Média dos F1-scores: 0.97142 ± 0.00095
Média dos ROC AUC: 0.97081 ± 0.00098


#### Resultados

In [28]:
import pandas as pd

# Definir os modelos
Modelos = ['Decision Tree', 'Naive Bayes', 'MLP', 'KNN']

# Criar um DataFrame do Pandas com os valores das variáveis
df = pd.DataFrame({
    'Modelos': Modelos,
    'Acuracia (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(AcurracyPosBal, AcurracyDPPosBal)],
    'Precisão (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(PrecisionPosBal, PrecisionDPPosBal)],
    'Recall (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(RecallPosBal, RecallDPPosBal)],
    'F1 Score': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(F1_scorePosBal, F1_scoreDPPosBal)],
    'AUC RoC': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Roc_aucPosBal, Roc_aucDPPosBal)]
})

# Função para destacar o maior valor de cada coluna
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

# Aplicar a função de destaque apenas para as colunas numéricas
numeric_columns = df.columns[1:]
styled_df = df.style.apply(highlight_max, subset=numeric_columns)

# Definir a cor do texto para a tabela
styled_df.set_properties(**{'color': 'white'}, subset=pd.IndexSlice[:, :])

# Exibir o DataFrame estilizado no Jupyter Notebook
styled_df


,Modelos,Acuracia (DREBIN),Precisão (DREBIN),Recall (DREBIN),F1 Score,AUC RoC
0,Decision Tree,0.9810 ± 0.0006,0.9737 ± 0.0011,0.9887 ± 0.0008,0.9812 ± 0.0006,0.9810 ± 0.0006
1,Naive Bayes,0.7299 ± 0.0046,0.7255 ± 0.0047,0.7397 ± 0.0141,0.7325 ± 0.0067,0.7299 ± 0.0046
2,MLP,0.9348 ± 0.0028,0.9329 ± 0.0085,0.9371 ± 0.0122,0.9349 ± 0.0032,0.9348 ± 0.0028
3,KNN,0.9708 ± 0.0010,0.9517 ± 0.0017,0.9920 ± 0.0014,0.9714 ± 0.0009,0.9708 ± 0.0010


### Bagging após o Balanceamento

#### Decision Tree

In [29]:
# Definir o classificador Decision Tree
modelo_arvore = DecisionTreeClassifier(criterion='gini')

# Definir o classificador de Bagging com Decision Tree como base
bagging_clf = BaggingClassifier(estimator = modelo_arvore, n_estimators=30)

# Treinar o modelo Bagging
bagging_clf.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = bagging_clf.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com Decision Tree:", accuracy_bagging)
print("Precisão do modelo Bagging com Decision Tree:", precision_bagging)
print("Recall do modelo Bagging com Decision Tree:", recall_bagging)
print("F1-score do modelo Bagging com Decision Tree:", f1_score_bagging)
print("AUC ROC do modelo Bagging com Decision Tree:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com Decision Tree:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBaggingPosBal.append(accuracy_bagging)
PrecisionBaggingPosBal.append(precision_bagging)
RecallBaggingPosBal.append(recall_bagging)
F1BaggingPosBal.append(f1_score_bagging)
Roc_aucBaggingPosBal.append(auc_roc_bagging)



Acurácia do modelo Bagging com Decision Tree: 0.9852375359442712
Precisão do modelo Bagging com Decision Tree: 0.9800849495111396
Recall do modelo Bagging com Decision Tree: 0.9906038637560245
F1-score do modelo Bagging com Decision Tree: 0.9853163333131911
AUC ROC do modelo Bagging com Decision Tree: 0.9852375359442712
Matriz de confusão do modelo Bagging com Decision Tree:
[[24194   497]
 [  232 24459]]


#### Naive Bayes

In [30]:
# Inicializar o modelo de Naive Bayes com distribuição de Gaussiana
modelo_naive_bayes = GaussianNB()

# Definir o classificador de Bagging com Naive Bayes como base
bagging_clf = BaggingClassifier(estimator=modelo_naive_bayes, n_estimators=30)

# Treinar o modelo Bagging
bagging_clf.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = bagging_clf.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com Naive Bayes:", accuracy_bagging)
print("Precisão do modelo Bagging com Naive Bayes:", precision_bagging)
print("Recall do modelo Bagging com Naive Bayes:", recall_bagging)
print("F1-score do modelo Bagging com Naive Bayes:", f1_score_bagging)
print("AUC ROC do modelo Bagging com Naive Bayes:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com Naive Bayes:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBaggingPosBal.append(accuracy_bagging)
PrecisionBaggingPosBal.append(precision_bagging)
RecallBaggingPosBal.append(recall_bagging)
F1BaggingPosBal.append(f1_score_bagging)
Roc_aucBaggingPosBal.append(auc_roc_bagging)


Acurácia do modelo Bagging com Naive Bayes: 0.7307925964926492
Precisão do modelo Bagging com Naive Bayes: 0.7253707731856832
Recall do modelo Bagging com Naive Bayes: 0.7428212709084282
F1-score do modelo Bagging com Naive Bayes: 0.7339923163118297
AUC ROC do modelo Bagging com Naive Bayes: 0.7307925964926492
Matriz de confusão do modelo Bagging com Naive Bayes:
[[17747  6944]
 [ 6350 18341]]


#### MLP

In [31]:
# Definir o classificador base (MLP)
modelo_mlp = MLPClassifier(hidden_layer_sizes=(100), max_iter=1000)

# Definir o classificador de Bagging com MLP como base
bagging_clf = BaggingClassifier(estimator=modelo_mlp, n_estimators=30)

# Treinar o modelo Bagging
bagging_clf.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = bagging_clf.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com MLP:", accuracy_bagging)
print("Precisão do modelo Bagging com MLP:", precision_bagging)
print("Recall do modelo Bagging com MLP:", recall_bagging)
print("F1-score do modelo Bagging com MLP:", f1_score_bagging)
print("AUC ROC do modelo Bagging com MLP:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com MLP:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBaggingPosBal.append(accuracy_bagging)
PrecisionBaggingPosBal.append(precision_bagging)
RecallBaggingPosBal.append(recall_bagging)
F1BaggingPosBal.append(f1_score_bagging)
Roc_aucBaggingPosBal.append(auc_roc_bagging)


Acurácia do modelo Bagging com MLP: 0.9441496901705074
Precisão do modelo Bagging com MLP: 0.9423581138316325
Recall do modelo Bagging com MLP: 0.9461747195334332
F1-score do modelo Bagging com MLP: 0.944262560122873
AUC ROC do modelo Bagging com MLP: 0.9441496901705075
Matriz de confusão do modelo Bagging com MLP:
[[23262  1429]
 [ 1329 23362]]


#### KNN

In [32]:
#inicializar o modelo KNN
modelo_knn = KNeighborsClassifier(n_neighbors=7)  # Escolha o número de vizinhos desejado

# Definir o classificador de Bagging com KNN como base
bagging_clf = BaggingClassifier(estimator=modelo_knn, n_estimators=30)

# Treinar o modelo Bagging
bagging_clf.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = bagging_clf.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com KNN:", accuracy_bagging)
print("Precisão do modelo Bagging com KNN:", precision_bagging)
print("Recall do modelo Bagging com KNN:", recall_bagging)
print("F1-score do modelo Bagging com KNN:", f1_score_bagging)
print("AUC ROC do modelo Bagging com KNN:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com KNN:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBaggingPosBal.append(accuracy_bagging)
PrecisionBaggingPosBal.append(precision_bagging)
RecallBaggingPosBal.append(recall_bagging)
F1BaggingPosBal.append(f1_score_bagging)
Roc_aucBaggingPosBal.append(auc_roc_bagging)


Acurácia do modelo Bagging com KNN: 0.9712648333400834
Precisão do modelo Bagging com KNN: 0.9511476426799007
Recall do modelo Bagging com KNN: 0.9935604066258961
F1-score do modelo Bagging com KNN: 0.9718915278410554
AUC ROC do modelo Bagging com KNN: 0.9712648333400834
Matriz de confusão do modelo Bagging com KNN:
[[23431  1260]
 [  159 24532]]


#### Resultados

In [33]:
Modelos = ['Decision Tree', 'Naive Bayes', 'MLP', 'KNN']
# Criar um DataFrame do Pandas com os valores das variáveis
dfB = pd.DataFrame({'Modelos Bagging': Modelos, 'Acuracia (DREBIN)': AcurracyBaggingPosBal,
                    'Precisão (DREBIN)': PrecisionBaggingPosBal, 'Recall (DREBIN)':RecallBaggingPosBal, 
                    'F1 Score': F1BaggingPosBal, 'AUC ROC': Roc_aucBaggingPosBal})

print(tabulate(dfB, headers='keys', tablefmt='fancy_grid', floatfmt=".4f"))

╒════╤═══════════════════╤═════════════════════╤═════════════════════╤═══════════════════╤════════════╤═══════════╕
│    │ Modelos Bagging   │   Acuracia (DREBIN) │   Precisão (DREBIN) │   Recall (DREBIN) │   F1 Score │   AUC ROC │
╞════╪═══════════════════╪═════════════════════╪═════════════════════╪═══════════════════╪════════════╪═══════════╡
│  0 │ Decision Tree     │              0.9852 │              0.9801 │            0.9906 │     0.9853 │    0.9852 │
├────┼───────────────────┼─────────────────────┼─────────────────────┼───────────────────┼────────────┼───────────┤
│  1 │ Naive Bayes       │              0.7308 │              0.7254 │            0.7428 │     0.7340 │    0.7308 │
├────┼───────────────────┼─────────────────────┼─────────────────────┼───────────────────┼────────────┼───────────┤
│  2 │ MLP               │              0.9441 │              0.9424 │            0.9462 │     0.9443 │    0.9441 │
├────┼───────────────────┼─────────────────────┼─────────────────────┼──

### Resultados

In [34]:
import pandas as pd

# Definir os modelos
Modelos = ['Decision Tree', 'Naive Bayes', 'MLP', 'KNN']

# Criar um DataFrame do Pandas com os valores das variáveis
df = pd.DataFrame({
    'Modelos': Modelos,
    'Acuracia (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Acurracy, AcurracyDP)],
    'Precisão (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Precision, PrecisionDP)],
    'Recall (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Recall, RecallDP)],
    'F1 Score': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(F1_score, F1_scoreDP)],
    'AUC RoC': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Roc_auc, Roc_aucDP)]
})

# Função para destacar o maior valor de cada coluna
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

# Aplicar a função de destaque apenas para as colunas numéricas
numeric_columns = df.columns[1:]
styled_df = df.style.apply(highlight_max, subset=numeric_columns)

# Definir a cor do texto para a tabela
styled_df.set_properties(**{'color': 'white'}, subset=pd.IndexSlice[:, :])

# Exibir o DataFrame estilizado no Jupyter Notebook
styled_df


,Modelos,Acuracia (DREBIN),Precisão (DREBIN),Recall (DREBIN),F1 Score,AUC RoC
0,Decision Tree,0.9817 ± 0.0007,0.7712 ± 0.0105,0.8194 ± 0.0120,0.7945 ± 0.0077,0.9042 ± 0.0059
1,Naive Bayes,0.9163 ± 0.0030,0.2501 ± 0.0098,0.4704 ± 0.0149,0.3264 ± 0.0097,0.7034 ± 0.0070
2,MLP,0.9769 ± 0.0009,0.8171 ± 0.0407,0.6031 ± 0.0410,0.6918 ± 0.0158,0.7984 ± 0.0195
3,KNN,0.9812 ± 0.0007,0.8112 ± 0.0128,0.7347 ± 0.0145,0.7709 ± 0.0095,0.8635 ± 0.0071


In [35]:
import pandas as pd
array = [0,0,0,0,0]
# Definir os modelos
Modelos = ['Decision Tree', 'Naive Bayes', 'MLP', 'KNN']

# Criar um DataFrame do Pandas com os valores das variáveis
df = pd.DataFrame({
    'Modelos': Modelos,
    'Acuracia (DREBIN)': [f"{media:.4f}" for media, dp in zip(AcurracyBagging, array)],
    'Precisão (DREBIN)': [f"{media:.4f}" for media, dp in zip(PrecisionBagging, array)],
    'Recall (DREBIN)': [f"{media:.4f}" for media, dp in zip(RecallBagging, array)],
    'F1 Score': [f"{media:.4f}" for media, dp in zip(F1Bagging, array)],
    'AUC RoC': [f"{media:.4f}" for media, dp in zip(Roc_aucBagging, array)]
})

# Função para destacar o maior valor de cada coluna
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

# Aplicar a função de destaque apenas para as colunas numéricas
numeric_columns = df.columns[1:]
styled_df = df.style.apply(highlight_max, subset=numeric_columns)

# Definir a cor do texto para a tabela
styled_df.set_properties(**{'color': 'white'}, subset=pd.IndexSlice[:, :])

# Exibir o DataFrame estilizado no Jupyter Notebook
styled_df


,Modelos,Acuracia (DREBIN),Precisão (DREBIN),Recall (DREBIN),F1 Score,AUC RoC
0,Decision Tree,0.9877,0.8994,0.8040,0.8490,0.9000
1,Naive Bayes,0.9178,0.2486,0.4487,0.3200,0.6938
2,MLP,0.9770,0.8281,0.5890,0.6884,0.7918
3,KNN,0.9816,0.8188,0.7356,0.7750,0.8641


In [36]:
import pandas as pd

# Definir os modelos
Modelos = ['Decision Tree', 'Naive Bayes', 'MLP', 'KNN']

# Criar um DataFrame do Pandas com os valores das variáveis
df = pd.DataFrame({
    'Modelos': Modelos,
    'Acuracia (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(AcurracyPosBal, AcurracyDPPosBal)],
    'Precisão (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(PrecisionPosBal, PrecisionDPPosBal)],
    'Recall (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(RecallPosBal, RecallDPPosBal)],
    'F1 Score': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(F1_scorePosBal, F1_scoreDPPosBal)],
    'AUC RoC': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Roc_aucPosBal, Roc_aucDPPosBal)]
})

# Função para destacar o maior valor de cada coluna
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

# Aplicar a função de destaque apenas para as colunas numéricas
numeric_columns = df.columns[1:]
styled_df = df.style.apply(highlight_max, subset=numeric_columns)

# Definir a cor do texto para a tabela
styled_df.set_properties(**{'color': 'white'}, subset=pd.IndexSlice[:, :])

# Exibir o DataFrame estilizado no Jupyter Notebook
styled_df


,Modelos,Acuracia (DREBIN),Precisão (DREBIN),Recall (DREBIN),F1 Score,AUC RoC
0,Decision Tree,0.9810 ± 0.0006,0.9737 ± 0.0011,0.9887 ± 0.0008,0.9812 ± 0.0006,0.9810 ± 0.0006
1,Naive Bayes,0.7299 ± 0.0046,0.7255 ± 0.0047,0.7397 ± 0.0141,0.7325 ± 0.0067,0.7299 ± 0.0046
2,MLP,0.9348 ± 0.0028,0.9329 ± 0.0085,0.9371 ± 0.0122,0.9349 ± 0.0032,0.9348 ± 0.0028
3,KNN,0.9708 ± 0.0010,0.9517 ± 0.0017,0.9920 ± 0.0014,0.9714 ± 0.0009,0.9708 ± 0.0010


In [37]:
import pandas as pd
array = [0,0,0,0,0]
# Definir os modelos
Modelos = ['Decision Tree', 'Naive Bayes', 'MLP', 'KNN']

# Criar um DataFrame do Pandas com os valores das variáveis
df = pd.DataFrame({
    'Modelos': Modelos,
    'Acuracia (DREBIN)': [f"{media:.4f}" for media, dp in zip(AcurracyBaggingPosBal, array)],
    'Precisão (DREBIN)': [f"{media:.4f}" for media, dp in zip(PrecisionBaggingPosBal, array)],
    'Recall (DREBIN)': [f"{media:.4f}" for media, dp in zip(RecallBaggingPosBal, array)],
    'F1 Score': [f"{media:.4f}" for media, dp in zip(F1BaggingPosBal, array)],
    'AUC RoC': [f"{media:.4f}" for media, dp in zip(Roc_aucBaggingPosBal, array)]    
})

# Função para destacar o maior valor de cada coluna
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

# Aplicar a função de destaque apenas para as colunas numéricas
numeric_columns = df.columns[1:]
styled_df = df.style.apply(highlight_max, subset=numeric_columns)

# Definir a cor do texto para a tabela
styled_df.set_properties(**{'color': 'white'}, subset=pd.IndexSlice[:, :])

# Exibir o DataFrame estilizado no Jupyter Notebook
styled_df


,Modelos,Acuracia (DREBIN),Precisão (DREBIN),Recall (DREBIN),F1 Score,AUC RoC
0,Decision Tree,0.9852,0.9801,0.9906,0.9853,0.9852
1,Naive Bayes,0.7308,0.7254,0.7428,0.7340,0.7308
2,MLP,0.9441,0.9424,0.9462,0.9443,0.9441
3,KNN,0.9713,0.9511,0.9936,0.9719,0.9713


In [105]:
import pandas as pd

# Definir os modelos
Modelos = ['Decision Tree DD', 'Naive Bayes DD', 'MLP DD', 'KNN DD',
            'Bagging Decision Tree DD', 'Bagging Naive Bayes DD', 'Bagging MLP DD', 'Bagging KNN DD',
              'Decision Tree DB', 'Naive Bayes DB', 'MLP DB', 'KNN DB', 
              'Bagging Decision Tree DB', 'Bagging Naive Bayes DB', 'Bagging MLP DB', 'Bagging KNN DB']


AcurracyAll = Acurracy + AcurracyBagging + AcurracyPosBal + AcurracyBaggingPosBal
PrecisionAll = Precision + PrecisionBagging + PrecisionPosBal + PrecisionBaggingPosBal
RecallAll = Recall + RecallBagging + RecallPosBal + RecallBaggingPosBal
F1_scoreAll = F1_score + F1Bagging + F1_scorePosBal + F1BaggingPosBal
Roc_aucAll = Roc_auc + Roc_aucBagging + Roc_aucPosBal + Roc_aucBaggingPosBal
AcurracyDPAll = AcurracyDP + [0,0,0,0] + AcurracyDPPosBal + [0,0,0,0]
PrecisionDPAll = PrecisionDP + [0,0,0,0] + PrecisionDPPosBal + [0,0,0,0] 
RecallDPAll = RecallDP + [0,0,0,0] + RecallDPPosBal + [0,0,0,0]
F1_scoreDPAll = F1_scoreDP + [0,0,0,0] + F1_scoreDPPosBal + [0,0,0,0]
Roc_aucDPAll = Roc_aucDP + [0,0,0,0] + Roc_aucDPPosBal + [0,0,0,0]

# print(len(AcurracyAll), len(PrecisionAll), len(RecallAll), len(F1_scoreAll), len(Roc_aucAll), len(AcurracyDPAll), 
#       len(PrecisionDPAll), len(RecallDPAll), len(F1_scoreDPAll), len(Roc_aucDPAll))


# Criar um DataFrame do Pandas com os valores das variáveis
df = pd.DataFrame({
    'Modelos': Modelos,
    'Acuracia': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(AcurracyAll, AcurracyDPAll)],
    'Precisão': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(PrecisionAll, PrecisionDPAll)],
    'Recall': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(RecallAll, RecallDPAll)],
    'F1 Score': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(F1_scoreAll, F1_scoreDPAll)],
    'AUC RoC': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Roc_aucAll, Roc_aucDPAll)]
})

# Aplicar a função de destaque apenas para as colunas numéricas
numeric_columns = df.columns[1:]
highlight_colors = ['green', 'blue', 'red']
def highlight_top_n_with_colors(s, n=3, colors=highlight_colors):
    sorted_index = np.argsort(s.values)[::-1]
    top_n_index = sorted_index[:n]    
    return [f'background-color: {colors[np.where(top_n_index == index)[0][0]]}' if index in top_n_index else '' for index in s.index]

styled_df = df.style.apply(highlight_top_n_with_colors, subset=numeric_columns)

# Definir a cor do texto para a tabela
styled_df.set_properties(**{'color': 'white'}, subset=pd.IndexSlice[:, :])

# Exibir o DataFrame estilizado no Jupyter Notebook
styled_df

,Modelos,Acuracia,Precisão,Recall,F1 Score,AUC RoC
0,Decision Tree DD,0.9817 ± 0.0007,0.7712 ± 0.0105,0.8194 ± 0.0120,0.7945 ± 0.0077,0.9042 ± 0.0059
1,Naive Bayes DD,0.9163 ± 0.0030,0.2501 ± 0.0098,0.4704 ± 0.0149,0.3264 ± 0.0097,0.7034 ± 0.0070
2,MLP DD,0.9769 ± 0.0009,0.8171 ± 0.0407,0.6031 ± 0.0410,0.6918 ± 0.0158,0.7984 ± 0.0195
3,KNN DD,0.9812 ± 0.0007,0.8112 ± 0.0128,0.7347 ± 0.0145,0.7709 ± 0.0095,0.8635 ± 0.0071
4,Bagging Decision Tree DD,0.9877 ± 0.0000,0.8994 ± 0.0000,0.8040 ± 0.0000,0.8490 ± 0.0000,0.9000 ± 0.0000
5,Bagging Naive Bayes DD,0.9178 ± 0.0000,0.2486 ± 0.0000,0.4487 ± 0.0000,0.3200 ± 0.0000,0.6938 ± 0.0000
6,Bagging MLP DD,0.9770 ± 0.0000,0.8281 ± 0.0000,0.5890 ± 0.0000,0.6884 ± 0.0000,0.7918 ± 0.0000
7,Bagging KNN DD,0.9816 ± 0.0000,0.8188 ± 0.0000,0.7356 ± 0.0000,0.7750 ± 0.0000,0.8641 ± 0.0000
8,Decision Tree DB,0.9810 ± 0.0006,0.9737 ± 0.0011,0.9887 ± 0.0008,0.9812 ± 0.0006,0.9810 ± 0.0006
9,Naive Bayes DB,0.7299 ± 0.0046,0.7255 ± 0.0047,0.7397 ± 0.0141,0.7325 ± 0.0067,0.7299 ± 0.0046
